In [ ]:
#Install all the required libraries(Please restart the run time after installation)
!pip install streamlit
!pip install apikey yfinance yahooquery
!pip install googlesearch-python
!pip install GoogleNews==1.6.6
!pip install newspaper3k==0.2.8
!pip install requests==2.28.1
!pip install fake_useragent==1.1.1
!pip install nltk==3.8.1
!pip install langchain chromadb
!pip install openai==0.28
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 44.1 MB/s eta 0:00:00
  Created wheel for apikey: filename=apikey-0.2.4-py3-none-any.whl size=6670 sha256=8850e00a9a848cfc6a883f26be3a0ab21f6879a9eee5b183eebd02a8095fc0d4
  Stored in directory: /root/.cache/pip/wheels/d0/b2/c9/a4400b26c52c13f16c796d15694407a8c610a3098b9e886651
Successfully built apikey
  Attempting uninstall: pandas


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.125.78.73


In [ ]:
import os
import requests
import json
import yfinance as yf
from yahooquery import Ticker
from googlesearch import search
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
import streamlit as st
import uuid
from fake_useragent import UserAgent
import time
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def get_ticker(company_name):
  """Function to convert a given companies name into its ticker"""
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "country": "United States"}
    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()
    company_ticker = data['quotes'][0]['symbol']
    return company_ticker
company_symbol = get_ticker("Google")

In [ ]:
%%writefile app1.py
"""Used for the streamlit application(Web interface)"""
import streamlit as st
import os
import requests
import json
import uuid
import yfinance as yf
from bs4 import BeautifulSoup
from newspaper import Article
from fake_useragent import UserAgent
import nltk
from newspaper import Article
import time
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import pandas as pd
import plotly.express as px

nltk.download('punkt')

st.title("Financial Stock Advisor")
company_name = st.text_input("Company stock to be analysed")
os.environ["OPENAI_API_KEY"] = st.text_input("Enter the open Api Key")

if company_name and os.environ["OPENAI_API_KEY"]:
    def get_ticker(comp_name):
        yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
        user_agent = 'Chrome/108.0.0.0 Safari/537.36'
        params = {"q": comp_name, "country": "United States"}
        res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
        data = res.json()
        company_ticker = data['quotes'][0]['symbol']
        return company_ticker

    company_symbol = get_ticker(company_name)

    if company_symbol:
        def get_stock_evolution(company_ticker, period="10d"):
            # Get the stock information
            stock = yf.Ticker(company_ticker)

            # Get historical market data
            hist = stock.history(period=period)

            # Generate a random name using uuid for investment file
            random_filename_investment = str(uuid.uuid4())[:8]
            investment_filename = f"investment_{random_filename_investment}.txt"

            # Append the string to the generated investment file
            with open(investment_filename, "w") as file:
                file.write(f"\nStock history for {company_ticker}:\n")
                file.write(hist.to_string())
                file.write("\n")

            return hist, investment_filename

        stock_data, investment_filename = get_stock_evolution(company_symbol, '10d')

        # Display the stock chart
        st.subheader(f"Stock Chart for {company_name} (Last 10 days)")
        fig = px.line(stock_data, x=stock_data.index, y='Close', title=f'{company_name} Stock Closing Prices')
        st.plotly_chart(fig)

        ua = UserAgent()

        def get_stock_news(company_symbol, period):
            headers = {'User-Agent': ua.random}

            headlines_data = []
            processed_count = 0

            search_url = f'https://www.bing.com/news/search?q={company_symbol} stock&qft=interval%1m"{period}"'

            try:
                response = requests.get(search_url, headers=headers)
                soup = BeautifulSoup(response.text, 'html.parser')
                news_links = soup.find_all('a', {'class': 'title'})

                # Generate a random name using uuid for headlines file
                random_filename_headlines = str(uuid.uuid4())[:8]
                headlines_filename = f"headlines_{random_filename_headlines}.txt"

                for link in news_links:
                    url = link['href']
                    try:
                        article = Article(url)
                        article.download()
                        article.parse()
                        article.nlp()

                        # Check if the article contains a relevant title with at least 5 words
                        if article.title and len(article.title.split()) >= 5:
                            # Extract the first three sentences from the article's text
                            summary = '. '.join(article.text.split('.')[:4]) + '.'

                            # Check if the extracted text has more than 5 words
                            if len(summary.split()) > 5:
                                headline_with_summary = f"{article.title}\n{summary}\n"
                                headlines_data.append(headline_with_summary)
                                processed_count += 1
                                print(f"Headlines scraped: {processed_count}")

                                if processed_count == 7:  # Stop after collecting the top 7 headlines
                                    break
                            else:
                                print(f"Skipping {url} as the extracted text has less than 5 words.")
                        else:
                            print(f"Skipping {url} as it doesn't contain a valid title with at least 5 words.")

                    except Exception as e:
                        print(f"Error processing {url}: {str(e)}")

                    time.sleep(2)  # to avoid making too many requests in a short time

                with open(headlines_filename, 'w', encoding='utf-8') as f:
                    f.write(''.join(headlines_data))

                return headlines_filename

            except Exception as e:
                print(f"Error fetching search results for {company_symbol} stock: {str(e)}")

        headlines_filename = get_stock_news(company_symbol, 'last month')

        # Combine the investment and headlines files into a single document
        combined_filename = f"combined_{str(uuid.uuid4())[:8]}.txt"

        with open(combined_filename, 'w', encoding='utf-8') as combined_file:
            # Read the contents of the investment file
            with open(investment_filename, 'r', encoding='utf-8') as investment_file:
                combined_file.write(investment_file.read())

            # Read the contents of the headlines file
            with open(headlines_filename, 'r', encoding='utf-8') as headlines_file:
                combined_file.write(headlines_file.read())

        # Example usage of the combined document in text_splitter
        documents = []
        loader = TextLoader(combined_filename)
        documents.extend(loader.load())
        text_splitter = CharacterTextSplitter(chunk_size=30000, chunk_overlap=10)
        chunked_documents = text_splitter.split_documents(documents)
        text_splitter = CharacterTextSplitter(chunk_size=2500, chunk_overlap=10)
        chunked_documents = text_splitter.split_documents(documents)
        vectordb = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())
        retriever = vectordb.as_retriever()
        template = """SYSTEM: You are an intelligent financial advisor helping people investment decisions

        Question: {question}

        Based on the context, help in solving the question asked by people
        =============
        {context}
        =============

        Question: {question}
        Answer:"""
        qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo"),
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
             ),
            },
          )
        question = st.text_input("How can I help you with financial advising?")
        if st.button("Generate") and question:
          result = qa({"query": question})
          st.write(result["result"])


Overwriting app1.py


In [ ]:
def get_stock_evolution(company_ticker, period="10d"):
  """Function to obtain the stocks value for the last 10 days"""
            # Get the stock information
            stock = yf.Ticker(company_ticker)

            # Get historical market data
            hist = stock.history(period=period)

            # Generate a random name using uuid for investment file
            random_filename_investment = str(uuid.uuid4())[:8]
            investment_filename = f"investment_{random_filename_investment}.txt"

            # Append the string to the generated investment file
            with open(investment_filename, "w") as file:
                file.write(f"\nStock history for {company_ticker}:\n")
                file.write(hist.to_string())
                file.write("\n")

            return hist, investment_filename

stock_data, investment_filename = get_stock_evolution(company_symbol, '10d')

In [ ]:
from fake_useragent import UserAgent
ua = UserAgent()

def get_stock_news(company_symbol, period):
  """Function to extract the stock news(based on top 7 news), News headlines and texts having atleast 5 words are considered"""
            headers = {'User-Agent': ua.random}

            headlines_data = []
            processed_count = 0

            search_url = f'https://www.bing.com/news/search?q={company_symbol} stock&qft=interval%1m"{period}"'

            try:
                response = requests.get(search_url, headers=headers)
                soup = BeautifulSoup(response.text, 'html.parser')
                news_links = soup.find_all('a', {'class': 'title'})

                # Generate a random name using uuid for headlines file
                random_filename_headlines = str(uuid.uuid4())[:8]
                headlines_filename = f"headlines_{random_filename_headlines}.txt"

                for link in news_links:
                    url = link['href']
                    try:
                        article = Article(url)
                        article.download()
                        article.parse()
                        article.nlp()

                        # Check if the article contains a relevant title with at least 5 words
                        if article.title and len(article.title.split()) >= 5:
                            # Extract the first three sentences from the article's text
                            summary = '. '.join(article.text.split('.')[:4]) + '.'

                            # Check if the extracted text has more than 5 words
                            if len(summary.split()) > 5:
                                headline_with_summary = f"{article.title}\n{summary}\n"
                                headlines_data.append(headline_with_summary)
                                processed_count += 1
                                print(f"Headlines scraped: {processed_count}")

                                if processed_count == 7:  # Stop after collecting the top 7 headlines
                                    break
                            else:
                                print(f"Skipping {url} as the extracted text has less than 5 words.")
                        else:
                            print(f"Skipping {url} as it doesn't contain a valid title with at least 3 words.")

                    except Exception as e:
                        print(f"Error processing {url}: {str(e)}")

                    time.sleep(2)  # to avoid making too many requests in a short time

                with open(headlines_filename, 'w', encoding='utf-8') as f:
                    f.write(''.join(headlines_data))

                return headlines_filename

            except Exception as e:
                print(f"Error fetching search results for {company_symbol} stock: {str(e)}")

headlines_filename = get_stock_news(company_symbol, 'last month')


Error processing https://www.msn.com/en-us/money/markets/goog-stock-alert-alphabet-is-spending-money-in-all-the-wrong-places/ar-AA1kTGwM: name 'Article' is not defined
Error processing http://www.aastocks.com/tc/usq/quote/stock-news.aspx?symbol=GOOG: name 'Article' is not defined
Error processing https://www.cnyes.com/global/astock/html5chart.aspx?area=us&code=GOOG: name 'Article' is not defined
Error processing https://www.msn.com/en-us/money/topstocks/goog-stock-will-2024-be-alphabet-s-year-of-ai/ar-AA1kItiN: name 'Article' is not defined
Error processing https://finance.yahoo.com/news/atai-capital-used-alphabet-goog-100529630.html: name 'Article' is not defined
Error processing https://www.nasdaq.com/articles/1-artificial-intelligence-ai-stock-to-buy-hand-over-fist-and-1-to-make-disappear-from-your: name 'Article' is not defined
Error processing https://hk.finance.yahoo.com/news/6-top-rated-ai-stocks-230000462.html: name 'Article' is not defined
Error processing https://uk.sports.ya

In [ ]:
# Combine the investment and headlines files into a single document
combined_filename = f"combined_{str(uuid.uuid4())[:8]}.txt"

In [ ]:
with open(combined_filename, 'w', encoding='utf-8') as combined_file:
            # Read the contents of the investment file
            with open(investment_filename, 'r', encoding='utf-8') as investment_file:
                combined_file.write(investment_file.read())

            # Read the contents of the headlines file
            with open(headlines_filename, 'r', encoding='utf-8') as headlines_file:
                combined_file.write(headlines_file.read())

In [ ]:
#Loading the document
documents = []
loader = TextLoader(combined_filename)
documents.extend(loader.load())

In [ ]:
#Splitting and chunking the text file
text_splitter = CharacterTextSplitter(chunk_size=1200, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

In [ ]:
#Please input the api key
os.environ["OPENAI_API_KEY"] = "XXXX"

In [ ]:
#The vector embeddings are stored in Chroma DB
vectordb = Chroma.from_documents(documents, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
#Customize the template as per need
template = """SYSTEM: You are an intelligent financial advisor helping people with current investment

Question: {question}

Based on the stock history of last 10 days and stock news, provide answer in less than 3 sentences, if it is a good idea or not
=============
{context}
=============

Question: {question}
Answer:"""

In [ ]:
#Defining the Langchain to include the required components
qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo"),
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    },
)


In [ ]:
#Provide the question to be answered
question = "Is it a right decision to invest in Google?"
result1 = qa({"query": question})

In [ ]:
result1

{'query': 'Is it a right decision to invest in Google?',
 'result': "Based on the stock history of the last 30 days, Google's stock price has been relatively stable with some minor fluctuations. Additionally, there is no significant negative news regarding Google in the last month. Therefore, it could be considered a good decision to invest in Google."}

In [ ]:
#Enter the local ip address in the local tunnel
! streamlit run app1.py & npx localtunnel --port 8501

[..................] / rollbackFailedOptional: verb npm-session bcead042ae8315e


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.78.73:8501

npx: installed 22 in 1.905s
your url is: https://chubby-icons-turn.loca.lt
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
Skipping https://www.msn.com